In [1]:
import tensorflow as tf

import numpy as np
import os
import time

Download the data

In [7]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

path_to_file = 'DavidCopperfield.txt'

Read the data

First, look in the text:

In [8]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1992581 characters


In [9]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿The Project Gutenberg EBook of David Copperfield, by Charles Dickens

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project 


In [10]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

88 unique characters


Process the text

Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [11]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [12]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

﻿
T
h
e
 


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\ufeffThe Project Gutenberg EBook of David Copperfield, by Charles Dickens\r\n\r\nThis eBook is for the use of'
' anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it aw'
'ay or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online'
' at www.gutenberg.org\r\n\r\n\r\nTitle: David Copperfield\r\n\r\nAuthor: Charles Dickens\r\n\r\nRelease Date: Decem'
'ber, 1996  [Etext #766]\r\nPosting Date: November 24, 2009\r\nLast Updated: September 25, 2016\r\n\r\nLanguag'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [14]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [15]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\ufeffThe Project Gutenberg EBook of David Copperfield, by Charles Dickens\r\n\r\nThis eBook is for the use o'
Target data: 'The Project Gutenberg EBook of David Copperfield, by Charles Dickens\r\n\r\nThis eBook is for the use of'


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [16]:
# Batch size
#BATCH_SIZE = 128
##################################################
# Decrease the batch size to 64
##################################################
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
##################################################
# Decrease the Buffer size to 10000
# reduce the amount of memory and improve performance
##################################################
BUFFER_SIZE = 10000
#BUFFER_SIZE = 20000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;

tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)

tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [19]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
#embedding_dim = 256
##################################################
# Increase the input layer from 256 to 1024
##################################################
embedding_dim = 1024


# Number of RNN units
##################################################
# Reduce RNN units from 1024 to 512
# previously had increased to 2048, got better results with 512
##################################################
rnn_units = 512
#rnn_units = 2048

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
                        #Added Drop out
                        tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [21]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 88) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 1024)          90112     
_________________________________________________________________
gru (GRU)                    (64, None, 512)           2362368   
_________________________________________________________________
dropout (Dropout)            (64, None, 512)           0         
_________________________________________________________________
dense (Dense)                (64, None, 88)            45144     
Total params: 2,497,624
Trainable params: 2,497,624
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
#This gives us, at each timestep, a prediction of the next character index:
sampled_indices

array([81, 43, 14, 73, 47, 46, 57, 75, 30, 55, 27, 70,  3, 72, 13, 82, 24,
       28,  7, 14, 24, 87, 75, 61, 34, 13, 56, 83, 15, 41, 20, 78, 14, 76,
       40, 69, 84, 71, 19, 51, 74, 39, 16, 63, 56, 38, 75, 35, 81, 75, 65,
       45, 20, 62, 36, 62, 16, 47, 60,  6, 58, 33,  9,  8, 31, 30, 26, 31,
       53, 22, 76, 64, 74, 58, 19, 24, 29, 73, 27, 74, 43,  0, 69, 85, 48,
       36, 34, 23, 69, 28, 33, 66,  4, 22,  9, 84, 25,  7, 34, 43])

Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [25]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 88)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.479314


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [26]:
model.compile(optimizer='adam', loss=loss)

Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Execute the training

To keep training time reasonable, use 10 epochs to train the model. 

In [28]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
308/308 [==============================] - 17s 55ms/step - loss: 2.1276
Epoch 2/10
308/308 [==============================] - 17s 55ms/step - loss: 1.5713
Epoch 3/10
308/308 [==============================] - 17s 55ms/step - loss: 1.4410
Epoch 4/10
308/308 [==============================] - 17s 55ms/step - loss: 1.3773
Epoch 5/10
308/308 [==============================] - 17s 54ms/step - loss: 1.3381
Epoch 6/10
308/308 [==============================] - 17s 55ms/step - loss: 1.3083
Epoch 7/10
308/308 [==============================] - 17s 55ms/step - loss: 1.2848
Epoch 8/10
308/308 [==============================] - 17s 55ms/step - loss: 1.2663
Epoch 9/10
308/308 [==============================] - 17s 55ms/step - loss: 1.2510
Epoch 10/10
308/308 [==============================] - 17s 55ms/step - loss: 1.2385


In [29]:
history.history

{'loss': [2.1275956630706787,
  1.5712804794311523,
  1.441001057624817,
  1.3772908449172974,
  1.3381413221359253,
  1.308270812034607,
  1.284793496131897,
  1.2663325071334839,
  1.2510111331939697,
  1.238451361656189]}

Generate text

Restore the latest checkpoint
To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [30]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 1024)           90112     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 512)            2362368   
_________________________________________________________________
dropout_1 (Dropout)          (1, None, 512)            0         
_________________________________________________________________
dense_1 (Dense)              (1, None, 88)             45144     
Total params: 2,497,624
Trainable params: 2,497,624
Non-trainable params: 0
_________________________________________________________________


The prediction loop

The following code block generates the text:

It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

Get the prediction distribution of the next character using the start string and the RNN state.

Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [31]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [37]:
print(generate_text(model, start_string=u"ROMEO: "))

#print(generate_text(model, start_string=u"Betsey: "))

ROMEO: 51. 
OF Jane Heop now, pictures.
Mr. Dick knieved possessed that I should have consure what surelfi,’
spoke of your father as a young women, but we were so considerably. Her Traddles, allosted a long agress
tosey, now; occasionally there of my if he blocked in assured to her eyes to my time traveeling intent, to be the sea, when I found now,
and good horse and scrotled lay into a contention
in shortly and fine again.’

‘Except kisses name, sawed!’

‘My dear Agnes. My mother shouldre a remembrance) or Murdstone sorrow--upon a butcher in his wife
for its sate than was lost. I shall be a scraglerly!’

‘Rely hearty but,
Doctors’ feeling, and have anybody en mother. I a pretence of her face
when you papery to stupy myself there, in the
getoon up and I touch for me--leave that thing, than I had of my
Doctreme, to my readon! I only, that you moan away
a part and stipendinate and tarned thousand dinaken!

‘Sendincthor, that was gash d eyes, why you
was is the most interest
